# Start of the file
-   robust MAD normalization and clipping, k=25
-   early stopping analysis


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from collections import Counter
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import hdbscan
import os
import torch
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import DataLoader, Dataset
from sklearn.manifold import TSNE
# import pacmap
import pickle
from sklearn.model_selection import train_test_split

print(torch.__version__)
print(torch.cuda.is_available())  # Check if CUDA is available (if using GPU)
print(torch.cuda.device_count())  # Check how many GPUs are available
print(torch.cuda.get_device_name(0))  # Get the name of the first GPU
print(torch.cuda.current_device())  # Get the current GPU device

# Check if cuDNN is enabled
print(torch.backends.cudnn.enabled)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)


2.6.0+cu118
True
1
NVIDIA GeForce RTX 4060 Ti
0
True
cuda:0


# Preprocess

In [ ]:
# load the TESS data from pickle file files_dataframe.pickle

with open("files_dataframe.pickle", "rb") as f:
  files_dataframe = pickle.load(f)


In [ ]:
from scipy.stats import median_abs_deviation
from tqdm import tqdm

def preprocess_lcs_per_object(df, set='train'):
    """
    Preprocess TESS light curves with per-object normalization and uncertainty weighting.
    
    Key changes from original:
    1. Per-object normalization (median + robust MAD scaling)
    2. Inverse-variance weighting using uncertainties
    3. Outlier handling using robust statistics per object
    4. Preserves astrophysical information while normalizing scale
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe with columns: 'object_name', 'tess_flux', 'tess_uncert', 'relative_time'
    set : str
        Dataset identifier ('train', 'val', 'test')
    
    Returns
    -------
    oids_used : list
        Object IDs that passed preprocessing
    target_class : list
        Class labels (if available) or object names
    data : list of np.array
        Normalized time-flux pairs for each object
    uncertainties : list of np.array
        Scaled uncertainties (time-uncertainty pairs)
    weights : list of np.array
        Inverse-variance weights for each data point
    normalization_params : dict
        Parameters used for normalization (for inverse transformation)
    """
    
    # Clean data
    df = df.copy()
    df = df[~df['tess_flux'].isna() & ~df['tess_uncert'].isna()]
    
    oids = np.unique(df['object_name'])
    
    data = []
    uncertainties = []
    weights = []
    target_class = []
    oids_used = []
    normalization_params = {}  # Store for potential inverse transformation
    
    print(f"=== Preprocessing with per-object normalization ===")
    print(f"Number of objects: {len(oids)}")
    
    for oid in tqdm(oids, desc=f"Processing {set} objects"):
        lc = df[df['object_name'] == oid].copy()
        
        if len(lc) < 20:  # Minimum points requirement
            continue
        
        # 1. Handle extreme outliers per object using robust statistics
        # Use median and MAD instead of mean/std for outlier detection
        median_flux = np.median(lc['tess_flux'])
        mad_flux = median_abs_deviation(lc['tess_flux'], scale='normal')
        
        # Define outlier bounds (e.g., median ± 5*MAD)
        # k = 25 for extreme clipping
        
        lower_bound = median_flux - 25 * mad_flux
        upper_bound = median_flux + 25 * mad_flux
        
        # Clip extreme outliers (not all high/low values, just extremes)
        n_extreme = ((lc['tess_flux'] < lower_bound) | (lc['tess_flux'] > upper_bound)).sum()
        if n_extreme > 0:
            # Cap extreme values at bounds (preserves more data than removal)
            lc.loc[lc['tess_flux'] < lower_bound, 'tess_flux'] = lower_bound
            lc.loc[lc['tess_flux'] > upper_bound, 'tess_flux'] = upper_bound
        
        # 2. PER-OBJECT NORMALIZATION
        # Calculate robust normalization parameters
        robust_median = np.median(lc['tess_flux'])
        robust_mad = median_abs_deviation(lc['tess_flux'], scale='normal')
        
        # Handle edge case: very small or zero variability
        if robust_mad < 1e-10:
            # Use standard deviation as fallback
            robust_mad = np.std(lc['tess_flux'])
            if robust_mad < 1e-10:
                robust_mad = 1.0  # Avoid division by zero
        
        # Normalize flux: (flux - median) / robust_mad
        # This gives zero median, unit robust variance
        flux_normalized = (lc['tess_flux'] - robust_median) / robust_mad
        
        # Scale uncertainties by the same factor
        uncert_scaled = lc['tess_uncert'] / robust_mad
        
        # 3. INVERSE-VARIANCE WEIGHTING
        # Weight = 1/σ² (inverse variance)
        # Add small epsilon to avoid division by zero
        epsilon = 1e-10
        weights_raw = 1.0 / (uncert_scaled**2 + epsilon)
        
        # Normalize weights to have max weight = 1 (optional but helps training)
        if weights_raw.max() > 0:
            weights_normalized = weights_raw / weights_raw.max()
        else:
            weights_normalized = np.ones_like(weights_raw)
        
        # 4. Time normalization (relative to start)
        time_normalized = lc['relative_time'] - np.min(lc['relative_time'])
        
        # 5. Store normalization parameters for this object
        normalization_params[oid] = {
            'median': robust_median,
            'mad': robust_mad,
            'n_points': len(lc),
            'n_extreme_clipped': n_extreme
        }
        
        # 6. Subsampling to fixed length (if needed for your model)
        if len(lc) > 100:
            # IMPORTANT: Sample while preserving time order and uncertainties
            indices = np.linspace(0, len(lc)-1, 100, dtype=int)
            flux_normalized = flux_normalized.iloc[indices]
            time_normalized = time_normalized.iloc[indices]
            uncert_scaled = uncert_scaled.iloc[indices]
            weights_normalized = weights_normalized[indices]
        
        elif len(lc) < 30:  # Skip objects with too few points after potential subsampling
            continue
        
        # 7. Prepare output arrays
        # Data: [time, normalized_flux]
        data.append(np.column_stack([time_normalized.values, flux_normalized.values]))
        
        # Uncertainties: [time, scaled_uncertainty]
        uncertainties.append(np.column_stack([time_normalized.values, uncert_scaled.values]))
        
        # Weights: inverse-variance weights
        weights.append(weights_normalized)
        
        # Store object info
        oids_used.append(oid)
        target_class.append(oid)  # Or use actual class if available
        
        # Optional: Store class label if available in dataframe
        if 'class' in lc.columns:
            target_class[-1] = lc['class'].iloc[0]
    
    print(f"=== Preprocessing complete ===")
    print(f"Processed objects: {len(oids_used)} / {len(oids)}")
    print(f"Average points per object: {np.mean([len(d) for d in data]):.1f}")
    
    # Print summary statistics
    if normalization_params:
        medians = [p['median'] for p in normalization_params.values()]
        mads = [p['mad'] for p in normalization_params.values()]
        print(f"Median of medians: {np.median(medians):.2f}")
        print(f"Median MAD: {np.median(mads):.2f}")
        print(f"Total extreme values clipped: {sum([p['n_extreme_clipped'] for p in normalization_params.values()])}")
    
    return oids_used, target_class, data, uncertainties, weights, normalization_params


# Use the enhanced preprocessing
all_ids, all_labels, all_data, all_uncertainties, all_weights, normalization_params = preprocess_lcs_per_object(files_dataframe)

=== Preprocessing with per-object normalization ===
Number of objects: 3659


Processing train objects: 100%|██████████| 3659/3659 [00:19<00:00, 183.30it/s]

=== Preprocessing complete ===
Processed objects: 3593 / 3659
Average points per object: 46.2
Median of medians: 193.28
Median MAD: 983.81
Total extreme values clipped: 287


In [56]:
count = 1
for key in normalization_params.keys():
    if normalization_params[key]['n_extreme_clipped'] > 0:  
        count += 1

print(f"Number of objects with extreme clipping: {count}")

Number of objects with extreme clipping: 74


In [10]:
class UncertaintyAwareLCDataset(Dataset):
    def __init__(self, data, uncertainties, weights, targets, ids, transform=None):
        self.data = data
        self.uncertainties = uncertainties
        self.weights = weights
        self.targets = targets
        self.ids = ids
        self.lengths = [len(seq) for seq in data]
        self.transform = transform

    def __getitem__(self, index):
        data = self.data[index]
        uncertainty = self.uncertainties[index]
        weights = self.weights[index]
        targets = self.targets[index]
        ids = self.ids[index]
        lengths = self.lengths[index]

        if self.transform:
            data = self.transform(data)
            uncertainty = self.transform(uncertainty)

        return data, uncertainty, weights, targets, ids, lengths

    def __len__(self):
        return len(self.data)

def uncertainty_collate_fn(batch):
    """Enhanced collate function that handles uncertainties and weights"""
    
    data, uncertainties, weights, targets, ids, lengths = zip(*batch)

    # Convert sequences to tensors and pad them
    sequences = [torch.tensor(d, dtype=torch.float32) for d in data]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    
    # Pad uncertainties
    uncertainty_sequences = [torch.tensor(u, dtype=torch.float32) for u in uncertainties]
    padded_uncertainties = pad_sequence(uncertainty_sequences, batch_first=True)
    
    # Pad weights
    weight_sequences = [torch.tensor(w, dtype=torch.float32) for w in weights]
    padded_weights = pad_sequence(weight_sequences, batch_first=True)

    # Convert other data to tensors
    targets = torch.tensor(targets)
    ids = torch.tensor(ids)
    lengths = torch.tensor(lengths)

    return padded_sequences, padded_uncertainties, padded_weights, targets, ids, lengths

# Create enhanced datasets and data loaders
all_ds = UncertaintyAwareLCDataset(
    all_data, all_uncertainties, all_weights,
    [all_labels.index(id) for id in all_labels], 
    [all_ids.index(id) for id in all_ids]
)

all_loader = DataLoader(
    all_ds,
    batch_size=128,
    num_workers=0,
    shuffle=True,
    collate_fn=uncertainty_collate_fn,
    pin_memory=torch.cuda.is_available()
)

all_loader_no_shuffle = DataLoader(
    all_ds,
    batch_size=128,
    num_workers=0,
    shuffle=False,
    collate_fn=uncertainty_collate_fn,
    pin_memory=torch.cuda.is_available()
)

In [11]:
train_ids, val_ids, train_labels, val_labels, train_data, val_data, \
train_uncertainties, val_uncertainties, train_weights, val_weights = train_test_split(all_ids, all_labels, all_data, 
                                                    all_uncertainties, all_weights, test_size=0.1, random_state=42)

train_final_ids = [train_ids.index(id) for id in train_ids]
train_final_labels = [train_labels.index(id) for id in train_labels]

train_ds = UncertaintyAwareLCDataset(train_data, train_uncertainties, train_weights,
                                     [train_labels.index(id) for id in train_labels], [train_ids.index(id) for id in train_ids])
train_loader = DataLoader(
    train_ds,
    batch_size=128,
    num_workers=0,
    shuffle=True,
    collate_fn=uncertainty_collate_fn,
    pin_memory=torch.cuda.is_available()
)

val_ds = UncertaintyAwareLCDataset(val_data, val_uncertainties, val_weights,
                                   [val_labels.index(id) for id in val_labels], [val_ids.index(id) for id in val_ids] )
val_loader = DataLoader(
    val_ds,
    batch_size=128,
    num_workers=0,
    shuffle=True,
    collate_fn=uncertainty_collate_fn,
    pin_memory=torch.cuda.is_available()
)

# Early stopping

In [116]:
class EarlyStopping:

    def __init__(self, patience=5, min_delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How many epochs to wait after last time validation loss improved.
            min_delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            path (str): Path for the checkpoint to be saved to.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        torch.save(model.state_dict(), self.path)
        print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')


# training

In [117]:
from src.util import Encoder, Decoder

class UncertaintyAwareRNN_VAE(torch.nn.Module):
    """RNN-VAE that incorporates measurement uncertainties"""
    
    def __init__(self, input_size=2, hidden_size=64, latent_size=16, dropout=0.2):
        super(UncertaintyAwareRNN_VAE, self).__init__()
        self.device = device
        
        # dimensions
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.latent_size = latent_size
        self.num_layers = 4
        self.dropout = dropout
        
        self.enc = Encoder(input_size=input_size, hidden_size=hidden_size, 
                          num_layers=self.num_layers, dropout=self.dropout)
        
        self.dec = Decoder(
            input_size=latent_size,
            output_size=input_size,
            hidden_size=hidden_size,
            dropout=self.dropout,
            num_layers=self.num_layers,
        )
        
        self.fc21 = torch.nn.Linear(self.hidden_size, self.latent_size)
        self.fc22 = torch.nn.Linear(self.hidden_size, self.latent_size)
        self.fc3 = torch.nn.Linear(self.latent_size, self.hidden_size)
    
    def reparameterize(self, mu, logvar):
        if self.training:
            return mu + torch.randn(mu.shape).to(device)*torch.exp(0.5*logvar)
        else:
            return mu
    
    def forward(self, x, lengths, weights=None):
        batch_size, seq_len, feature_dim = x.shape
        
        # encode input space
        enc_output, enc_hidden = self.enc(x, lengths)
        
        # Correctly accessing the hidden state of the last layer
        enc_h = enc_hidden[-1].to(device)
        
        # extract latent variable z
        mu = self.fc21(enc_h)
        logvar = self.fc22(enc_h)
        z = self.reparameterize(mu, logvar)
        
        # initialize hidden state
        h_ = self.fc3(z)
        h_ = h_.unsqueeze(0)
        h_ = h_.repeat(self.dec.num_layers, 1, 1)
        
        # decode latent space
        z = z.repeat(1, seq_len, 1)
        z = z.view(batch_size, seq_len, self.latent_size).to(device)
        
        # initialize hidden state
        hidden = h_.contiguous()
        x_hat, hidden = self.dec(z, hidden)
        
        return x_hat, mu, logvar

def uncertainty_weighted_ELBO(x_hat, x, mu, logvar, weights=None):
    """ELBO loss with uncertainty weighting"""
    
    # Reconstruction loss with uncertainty weights
    if weights is not None:
        # Expand weights to match x shape if needed
        if weights.dim() == 2:  # [batch_size, seq_len]
            weights = weights.unsqueeze(-1)  # [batch_size, seq_len, 1]
        
        # Weighted MSE
        weighted_MSE = torch.sum(weights * (x_hat - x)**2)
    else:
        weighted_MSE = torch.nn.MSELoss(reduction='sum')(x_hat, x)
    
    # KL divergence (unchanged)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return weighted_MSE + KLD

## early stop training

In [ ]:
# no validation for now
from importlib.resources import path
from pathlib import Path
from src.util import save_checkpoint, plot_model_loss


early_stopping = EarlyStopping(patience=2)

hidden_size=128
latent_size=16
nepochs = 800
num_layers = 4

model1 = UncertaintyAwareRNN_VAE(hidden_size=hidden_size, latent_size=latent_size)
model1.to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=1.e-4)

training_losses = []
validation_losses = []

# This might ALSO take a bit of time to train.
print("Beginning training...")

for epoch in range(0, nepochs):
    # set to training mode
    model1.train()
    train_loss = 0

    for x, uncertainties, weights, labels, ids, lengths in train_loader:
        x = x.to(device)
        weights = weights.to(device)
        
        x_hat, mu, logvar = model1(x, lengths)
        loss = uncertainty_weighted_ELBO(x_hat, x, mu, logvar, weights)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_norm=1.0)
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader.dataset)
    training_losses.append(avg_train_loss)

    # Validation phase
    model1.eval()
    val_loss = 0
    
    with torch.no_grad():
        for x, uncertainties, weights, labels, ids, lengths in val_loader:
            x = x.to(device)
            weights = weights.to(device)
            
            x_hat, mu, logvar = model1(x, lengths)
            loss = uncertainty_weighted_ELBO(x_hat, x, mu, logvar, weights)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    validation_losses.append(avg_val_loss)

    print(f'Epoch {epoch} | Train: {avg_train_loss:.2f} | Val: {avg_val_loss:.2f}')

    # --- EARLY STOPPING CHECK ---
    # Call the object with the current validation loss
    early_stopping(avg_val_loss, model1)
    
    print(f'Epoch {epoch} | Train: {avg_train_loss:.2f} | Val: {avg_val_loss:.2f}')

# --- LOAD BEST MODEL ---
# After the loop breaks, load the best saved weights
model1.load_state_dict(torch.load(path))
print("Best model loaded.")

plot_model_loss(training_losses, validation_losses)

Beginning training...
Epoch 0 | Train: 8246.56 | Val: 7952.64
Validation loss decreased (7952.636285 --> 7952.636285).  Saving model ...
Epoch 0 | Train: 8246.56 | Val: 7952.64
Epoch 1 | Train: 6733.80 | Val: 5390.50
Validation loss decreased (5390.501910 --> 5390.501910).  Saving model ...
Epoch 1 | Train: 6733.80 | Val: 5390.50
Epoch 2 | Train: 4705.59 | Val: 4213.49
Validation loss decreased (4213.491840 --> 4213.491840).  Saving model ...
Epoch 2 | Train: 4705.59 | Val: 4213.49
Epoch 3 | Train: 3914.43 | Val: 3740.11
Validation loss decreased (3740.107031 --> 3740.107031).  Saving model ...
Epoch 3 | Train: 3914.43 | Val: 3740.11
Epoch 4 | Train: 3537.37 | Val: 3449.64
Validation loss decreased (3449.639844 --> 3449.639844).  Saving model ...
Epoch 4 | Train: 3537.37 | Val: 3449.64
Epoch 5 | Train: 3276.56 | Val: 3217.43
Validation loss decreased (3217.426042 --> 3217.426042).  Saving model ...
Epoch 5 | Train: 3276.56 | Val: 3217.43
Epoch 6 | Train: 3058.47 | Val: 3011.67
Validati

AttributeError: 'function' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.